### Базовое решение кейса "Рекомендательная система видео"
### Кейсодержатель: RUTUBE
#### Описание решения:

Приведенное базовое решение основано на подсчете совстречаемостей видео в сессии пользователя, или если говорить "по-бытовому": посчитаем как часто один товар попадает вместе с другим в одну корзину. На рисунке ниже вы можете видеть, что к пиву очень разумно рекомендовать рыбку, ведь ее частенько берут в дополнение к этому напитку.

Однако вполне реальной выглядит ситуация, что пиво встречается чаще в корзине с другим продуктом - с молоком. Почему так происходит? Молоко много с чем часто встречается в корзине просто потому что это очень популярный товар в магазине, в отличие от вяленой рыбки. Но было бы странно получать в рекомендациях к пиву молоко. Поэтому для расчета схожести применяется нормировка на популярность товаров! Далее на основе полученных для каждой пары товаров чисел, рекомендации можно отранжировать и получить в выдаче красоту, а не молоко:)

Подробнее читайте тут: https://www.inf.unibz.it/~ricci/ISR/papers/p293-davidson.pdf

![Рекомендации к пиву]

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# !pip install -r requirements.txt
# !pip install implicit # ALS
!pip install lightfm

In [3]:
import pandas as pd
from collections import OrderedDict

In [4]:
videos_df = pd.read_parquet('/content/gdrive/MyDrive/Colab Notebooks/rutube/clean_data/preFinal_video.parquet')

In [5]:
videos_df.head()

,item_id,video_title,author_title,season,video_description,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,...,category_title_Спорт,category_title_Спорт/Игры,category_title_Строительство и ремонт,category_title_Телепередачи,category_title_Техника и оборудование,category_title_Технологии и интернет,category_title_Фильмы,category_title_Хобби,category_title_Эзотерика,category_title_Юмор
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,-0.066471,В видео я обывательским взглядом расскажу про ...,0.307617,-0.146267,-0.169441,-0.108148,-0.199587,...,0,0,0,0,0,1,0,0,0,0
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",-0.066471,Видео с канала УЦ Академия безопасности (ab-dp...,0.033179,-0.222731,-0.168210,-0.108148,-0.199587,...,0,0,0,0,0,0,0,0,0,0
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,-0.066471,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,0.028254,-0.133251,-0.169346,-0.108148,-0.199587,...,0,0,0,0,0,0,0,0,0,0
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИ...,ПАПА ДОЧКИ Games,-0.066471,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookha...,-0.387065,-0.051790,-0.137591,-0.108148,-0.199587,...,0,0,0,0,0,0,0,0,0,0
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Б...","ЗАО ""Сплайн-Центр""",-0.066471,"10.08.2023 Вебинар ""Особенности трудоустройств...",0.567876,0.845478,-0.168541,-0.108148,-0.199587,...,0,0,0,0,0,0,0,0,0,0


In [6]:
feature_cols = videos_df.columns.tolist()
feature_cols.remove('item_id')

# Create a list to store item_features for each row
item_features_list = []

# Iterate over each row to extract item_features
for _, row in videos_df.iterrows():
    item_features = row[feature_cols].tolist()
    item_features_list.append(item_features)

# Create a new DataFrame with "item_id" and "item_features" columns
new_df = pd.DataFrame({
    'item_id': videos_df['item_id'],
    'item_features': item_features_list
})

In [8]:
del videos_df

In [ ]:
pst1_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rutube/clean_data/final_df_1.csv')
pst2_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rutube/clean_data/final_df_2.csv')
clean_pst_df = pd.concat([pst1_df, pst2_df], axis=0)
clean_pst_df = clean_pst_df.reset_index(drop=True)
del pst1_df
del pst2_df
pst3_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rutube/clean_data/final_df_3.csv')
pst4_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rutube/clean_data/final_df_4.csv')
clean_pst_df = pd.concat([clean_pst_df, pst3_df, pst4_df], axis=0)
clean_pst_df = clean_pst_df.reset_index(drop=True)
del pst3_df
del pst4_df

In [5]:
clean_pst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29963739 entries, 0 to 29963738
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   user_id       object 
 1   item_id       object 
 2   duration      int64  
 3   watch_time    float64
 4   is_autorized  int64  
 5   is_watched    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.3+ GB


In [ ]:
videos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2320660 entries, 0 to 2320659
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   item_id               object 
 1   video_title           object 
 2   author_title          object 
 3   tv_title              object 
 4   season                int64  
 5   video_description     object 
 6   category_title        object 
 7   publicated            object 
 8   duration              int64  
 9   channel_sub           int64  
 10  tv_sub                int64  
 11  ctr.CTR_10days_21_07  float64
 12  ctr.CTR_10days_01_08  float64
 13  ctr.CTR_10days_10_08  float64
 14  ctr.CTR_10days_21_08  float64
dtypes: float64(4), int64(4), object(7)
memory usage: 265.6+ MB


In [6]:
train_data = clean_pst_df[['user_id', 'item_id', 'is_watched']]

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29963739 entries, 0 to 29963738
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     object
 1   item_id     object
 2   is_watched  int64 
dtypes: int64(1), object(2)
memory usage: 685.8+ MB


# Получение разряжённой матрицы пользователей и видео для ALS

In [8]:
# from scipy import sparse

# user_ids = train_data['user_id'].unique()
# item_ids = train_data['item_id'].unique()
# user_id_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
# item_id_to_idx = {item_id: idx for idx, item_id in enumerate(item_ids)}

# rows = [user_id_to_idx[user_id] for user_id in train_data['user_id']]
# cols = [item_id_to_idx[item_id] for item_id in train_data['item_id']]

# sparse_user_item = sparse.csr_matrix((train_data['is_watched'].values, (rows, cols)), shape=(len(user_ids), len(item_ids)))

KeyboardInterrupt: ignored

In [ ]:
# sparse_user_item

<1313891x261288 sparse matrix of type '<class 'numpy.int64'>'
	with 2006839 stored elements in Compressed Sparse Row format>

# Обучение LightFM

In [9]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
import numpy as np

In [10]:
train_data.head()

,user_id,item_id,is_watched
0,user_15057892,video_1989987,1
1,user_8293675,video_331810,5
2,user_325285,video_523098,1
3,user_11178664,video_833222,1
4,user_26782853,video_863791,5


## Формирование датасета

In [11]:
train_data['user_id'] = train_data['user_id'].str.replace('user_', '').astype(int)
train_data['item_id'] = train_data['item_id'].str.replace('video_', '').astype(int)

<ipython-input-11-37ffda31e2fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['user_id'] = train_data['user_id'].str.replace('user_', '').astype(int)
<ipython-input-11-37ffda31e2fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['item_id'] = train_data['item_id'].str.replace('video_', '').astype(int)


In [12]:
dataset = Dataset()

users = train_data['user_id']
items = train_data['item_id']

# Fit the dataset with the user and item data
dataset.fit(train_data['user_id'], train_data['item_id'])

(interactions, _) = dataset.build_interactions(zip(train_data['user_id'], train_data['item_id'], train_data['is_watched']))

In [ ]:
# Split the dataset into training and testing sets
# train, test = random_train_test_split(interactions, test_percentage=0.2)

In [13]:
# Create a model object
model = LightFM(loss='warp')

# Train the model
model.fit(interactions, epochs=20, num_threads=2)

In [ ]:
import pickle

with open('/content/gdrive/MyDrive/Colab Notebooks/rutube/recsys.pickle', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
scores = model.predict(674229, np.arange(interactions.shape[1]))

In [15]:
top_items = np.argsort(-scores)[:1000]

In [ ]:
#recommendations = model.predict(np.arange(train_data['user_id'].shape[0]), np.arange(train_data['item_id'].shape[0]))

ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 1313891 vs 2439644.

In [ ]:
# # Создаем и обучаем модель
# model = LightFM(loss='warp')
# sparse_user_item_coo = sparse.coo_matrix((train_data['target'].values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
# %time model.fit(dataset, epochs=30, num_threads=2)

In [ ]:
# sparse_user_item_coo

In [ ]:
# model.predict(user_ids, item_ids)

In [ ]:
# # Предсказываем оценки для всех пар пользователь-видео
# user_ids = np.array(range(len(user_ids)))  # Convert range object to a list
# item_ids = np.array(range(len(item_ids)))  # Convert range object to a list

# scores = model.predict(user_ids, item_ids)
# top_recommendations = np.argsort(-scores, axis=1)[:, :10]

ValueError: Expected the number of user IDs (1313891) to equal the number of item IDs (261288)

# Обучение ALS

In [ ]:
# import implicit

# model = implicit.als.AlternatingLeastSquares(factors=16, regularization=0, iterations=20)
# model.fit(sparse_user_item)

In [ ]:
# user_ids = train_data['user_id'].unique()
# df_recommendations = pd.DataFrame(dense_recommendations, columns=user_ids).stack().reset_index()
# df_recommendations.columns = ['user_id', 'recommendation_position', 'recommendation_score']
# df_recommendations = df_recommendations.groupby('user_id')['recommendation_position'].apply(list).reset_index()

In [ ]:
user_id_dict = train_data['user_id'].to_dict()
item_id_dict = train_data['item_id'].to_dict()
user_id_dict = {val: key for key, val in user_id_dict.items()}

# Рекомендации по одному пользователю для ALS

In [ ]:
# filtered_counts = clean_pst_df['user_id'].value_counts().loc[lambda x: x.between(1, 1000)]

In [ ]:
# filtered_counts

In [ ]:
# user_id = 16540994
# user_id_matrix = user_id_dict[user_id]
# user_id_matrix

674229

In [ ]:
# recommendations = model.recommend(user_id_matrix, sparse_user_item[user_id_matrix], N=1000)

In [ ]:
history = list(OrderedDict.fromkeys(clean_pst_df[clean_pst_df['user_id'] == user_id]['item_id']))
history

In [ ]:
recommendations = [item_id_dict.get(key) for key in recommendations[0]]
unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)

In [ ]:
# %time
# user_id_matrix = user_id_dict[user_id]
# recommendations = model.recommend(user_id_matrix, sparse_user_item[user_id_matrix], N=1000)
# history = list(OrderedDict.fromkeys(s_pst_df[s_pst_df['user_id'] == user_id]['item_id']))
# recommendations = [item_id_dict.get(key) for key in recommendations[0]]
# unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)
# videos_df[videos_df['item_id'].isin(list(unique_list)[:10])]

# Рекомендации по одному пользователю для LightFM WARP

In [ ]:
recommendations = [item_id_dict.get(key) for key in top_items.tolist()]

In [ ]:
recommendations = [f"video_{video}" for video in recommendations]

In [ ]:
history = list(OrderedDict.fromkeys(s_pst_df[s_pst_df['user_id'] == user_id]['item_id']))
recommendations = [f"video_{video}" for video in recommendations]
unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)
videos_df[videos_df['item_id'].isin(list(unique_list)[:10])]

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
307013,video_1508131,Анастасия Попова («Бэби-тур») высказалась об о...,Вокруг•ТВ,None,0,В 2023 году актриса снялась в комедии положени...,Лайфстайл,2023-07-21 18:27:19+03:00,99883,2146,0,0.078947,0.010526,0.010309,0.000000
519031,video_1535803,"Экстрасенсы ведут расследование, 6 сезон, 16 в...",Экстрасенсы ведут расследование,Экстрасенсы ведут расследование,6,"Подмосковный Пушкино — почти Париж. И там, и т...",Телепередачи,2022-03-29 13:14:16+03:00,4270647,4973,0,0.243671,0.202864,0.204230,0.222222
956478,video_283933,"Выжить в Дубае, 9 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-20 21:00:23+03:00,5616280,75496,0,NaN,NaN,NaN,0.572971
1282940,video_796847,ИТАЛЬЯНСКИЕ КОЖАНЫЕ ВИНТАЖНЫЕ СУМКИ У ОКСАНЫ? ...,"""Модный Шопинг с Лилией и Ириной!""",None,0,Контакты ПРОДАВЦА: Линия 29 ПАВИЛЬОН 59 \n З...,Красота,2022-04-12 16:00:21+03:00,2959082,955,0,0.000000,0.000000,0.000000,0.111111
1409601,video_1905049,Татьяна Руденко: к новому ритму жизни должны п...,НТК Эфир,None,0,Гости программы «Детали» — психолог Татьяна Ру...,Телепередачи,2023-08-21 20:18:24+03:00,1559894,632,0,NaN,NaN,NaN,0.000000
1909813,video_1366097,brawlstars,Играю на чиле,None,0,None,Видеоигры,2023-05-05 13:03:32+03:00,97003,16,0,0.000000,0.000000,0.000000,0.000000
2017664,video_1156618,"ЖИТЬ ЖИЗНЬ 1,2,3,4,5,6,7,8 серия 1 сезон сериа...",ПОПКОРН,None,0,None,Лайфстайл,2023-04-21 15:52:00+03:00,3510277,498,0,0.201608,0.226143,0.266551,0.217391
2151746,video_500499,حيدر وليد الفصل الاول المحاضرة[29] دفعة 2023,مفتاح السادس Sixth Key,None,0,None,Сериалы,2023-08-17 02:16:16+03:00,1800416,552,0,NaN,NaN,NaN,0.050000


In [ ]:
def fill_submission(submission):
    for user_id in submission['user_id'].values:
        user_id_matrix = user_id_dict[user_id]
        recommendations = model.recommend(user_id_matrix, sparse_user_item[user_id_matrix], N=1000)
        history = list(OrderedDict.fromkeys(s_pst_df[s_pst_df['user_id'] == user_id]['item_id']))
        recommendations = [item_id_dict.get(key) for key in recommendations[0]]
        unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)
        submission[submission['user_id'] == user_id]['recs'] = list(unique_list)[:10]
    submission.to_csv("/kaggle/working/submission.csv", index=False)

fill_submission(pd.read_csv('/kaggle/input/name_data/submission.csv'))